In [600]:
import torch
import my_utils as mu
import pandas as pd
import torch.utils.data as data
from torch import nn

In [601]:
df = pd.read_csv('Fish.csv')
df

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [602]:
feats = torch.tensor(df.iloc[::,2:4].values)
labs = torch.tensor(df.iloc[::,1].values)

In [603]:
dataset = data.TensorDataset(feats,labs)
train_iter, test_iter = data.random_split(dataset,[109,50],generator=torch.Generator().manual_seed(42))
batch_size = 10
train_iter,test_iter = data.DataLoader(train_iter,batch_size),data.DataLoader(test_iter,batch_size)
data_iter = data.DataLoader(dataset,batch_size)
print(dataset[0])

(tensor([23.2000, 25.4000], dtype=torch.float64), tensor(242., dtype=torch.float64))


In [604]:
X,y = next(iter(data_iter))
X

tensor([[23.2000, 25.4000],
        [24.0000, 26.3000],
        [23.9000, 26.5000],
        [26.3000, 29.0000],
        [26.5000, 29.0000],
        [26.8000, 29.7000],
        [26.8000, 29.7000],
        [27.6000, 30.0000],
        [27.6000, 30.0000],
        [28.5000, 30.7000]], dtype=torch.float64)

In [605]:
num_inputs,num_outputs = 2,1
net = nn.Linear(num_inputs,num_outputs)

In [606]:
def init_weights(m):
    if isinstance(m, nn.Linear): # by checking the type we can init different layers in different ways
        torch.nn.init.normal_(m.weight, std=0.01)
        torch.nn.init.zeros_(m.bias)

In [607]:
net.apply(init_weights)

Linear(in_features=2, out_features=1, bias=True)

In [608]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.01)
loss = nn.MSELoss()
num_epochs=3

In [609]:
#mu.train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer)

In [610]:
X,y = next(iter(data_iter))
X.shape
y.shape

torch.Size([10])

In [611]:
X,y = next(iter(data_iter))
net(X.float()).view(-1)

tensor([-0.1178, -0.1221, -0.1244, -0.1354, -0.1344, -0.1393, -0.1393, -0.1382,
        -0.1382, -0.1402], grad_fn=<ViewBackward0>)

In [612]:
for epoch in range(num_epochs):
    for X,y in data_iter:
        y_hat = net(X.float())
        y_hat = y_hat.view(-1)
        #print(y_hat)
        l = loss(y_hat,y.float())
        #print(y_hat.view(-1).shape)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l = loss(net(feats.float()).view(-1),labs.float())
    print(f'epoch {epoch+1},loss {l:f}')

epoch 1,loss inf
epoch 2,loss nan
epoch 3,loss nan
